In [ ]:
!pip install munkres

In [ ]:
!pip install tensorflow==2.12.0rc0

In [ ]:
!git clone https://github.com/hira-husnain/SADSE

fatal: destination path 'SADSE' already exists and is not an empty directory.


In [ ]:
cd SADSE

/content/SADSE


In [ ]:
import numpy as np
x_train=np.load('./datasets/EYALEB/d_eyaleb.npy')
x_train=x_train.reshape(x_train.shape[0]*x_train.shape[1],784)
y=np.load('./datasets/EYALEB/l_eyaleb.npy')
y=y.reshape(y.shape[0]*y.shape[1],)
alpha_trans1=np.load('./datasets/EYALEB/Sb_eyaleb.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32')

In [ ]:
# import tensorflow as tf
# print(tf.__version__)

In [ ]:
import glob
import numpy as np
import random
#import tensorflow as tf
import os
from sklearn.utils import shuffle
batchSize=486
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
#from tensorflow.keras import backend as B
k_cluster = 38
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres


def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    for a in range (1):
        if uc1.shape==uc2.shape:
            m = np.ones([l1, l2])
            for i in range(l1):
                it_i = np.nonzero(c1 == uc1[i])[0]
                for j in range(l2):
                    it_j = np.nonzero(c2 == uc2[j])[0]
                    m_ij = np.intersect1d(it_j, it_i)
                    m[i,j] =  -m_ij.size
        else:
            print('assertion handeled')
            return
        #break
    return m


def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes)
    if cost_matrix is None:
        return (None,None)
    else:
        m = Munkres()
        indexes = m.compute(cost_matrix)
        mapper = { old: new for (old, new) in indexes }
        new_labels = translate_clustering(labels, mapper)
        new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
        return new_labels,accuracy(new_cm)

def arch():
    encoding_dim = 484
    input_img = Input(shape=(784,))
    encoded_layer_2 = Dense(700, activation='tanh')(input_img)
    encoded_layer_3 = Dense(600, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(500, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(500, activation='tanh')(encoded)
    decoded_layer_3 = Dense(600, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(700, activation='tanh')(decoded_layer_3)
    decoded = Dense(784, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder

def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T

    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + 1e-9)).T
    return q


In [ ]:
encoder,encoding_dim,autoencoder=arch()
print('loading trained model')
encoder.load_weights('./checkpoints/chk_eyaleb/sdsc_Eyaleb.ckpt')
z_test=encoder.predict(x_train)
ic= np.load('./checkpoints/chk_eyaleb/sdsc_icEyaleb.npy')
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y)
nmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
f1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')

loading trained model
76/76 [==============================] - 1s 8ms/step


In [ ]:
print(acc,nmi,f1,prec)

1.0 1.0 1.0 1.0


#modelling from scratch


In [ ]:
import numpy as np
import glob
import os
import cv2
import tensorflow.compat.v1 as tf
from sklearn.metrics.pairwise import euclidean_distances
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
dnet=tf.keras.applications.densenet.DenseNet201(include_top=False,weights="imagenet",input_tensor=None,input_shape=None,pooling=None)
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from munkres import Munkres
from sklearn.decomposition import PCA
pca = PCA(n_components=784)
k_cluster = 38
batchSize=486

74836368/74836368 [==============================] - 2s 0us/step


In [ ]:



def eig_vec(x_train):
    nev_trans=[]
    SortedEigVec=np.zeros((batchSize,batchSize))
    for l in range (0,len(x_train),batchSize):
        xtr=x_train[l:l+batchSize]
        d=(euclidean_distances(xtr,xtr))
        W=np.zeros((len(xtr),len(xtr)))
        for i in range(len(xtr)):
            for j in range (len(xtr)):
                if i==j:
                    W[i,j]=0
                else:
                    W[i,j]=np.exp(-(d[i,j]**2)/(2*0.09))
        degree=np.diag(sum(W))
        L = degree-W
        eig_val,eig_vec=np.linalg.eig(L)
        dec_eigVal=np.flip(np.sort(eig_val))
        indexArr=(np.flip(np.argsort(eig_val)))
        for i in range (len(indexArr)):
            SortedEigVec[:,i]=eig_vec[:,indexArr[i]]
        SortedEigVec=np.array(SortedEigVec)
        nev_trans.append(SortedEigVec)
    return nev_trans,L,eig_val,eig_vec,dec_eigVal,W

def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    for a in range (1):
        if uc1.shape==uc2.shape:
            m = np.ones([l1, l2])
            for i in range(l1):
                it_i = np.nonzero(c1 == uc1[i])[0]
                for j in range(l2):
                    it_j = np.nonzero(c2 == uc2[j])[0]
                    m_ij = np.intersect1d(it_j, it_i)
                    m[i,j] =  -m_ij.size
        else:
            print('assertion handeled')
            return
        #break
    return m


def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes)
    if cost_matrix is None:
        return (None,None)
    else:
        m = Munkres()
        indexes = m.compute(cost_matrix)
        mapper = { old: new for (old, new) in indexes }
        new_labels = translate_clustering(labels, mapper)
        new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
        return new_labels,accuracy(new_cm)

def arch():
    encoding_dim = 484
    input_img = Input(shape=(784,))
    encoded_layer_2 = Dense(700, activation='tanh')(input_img)
    encoded_layer_3 = Dense(600, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(500, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(500, activation='tanh')(encoded)
    decoded_layer_3 = Dense(600, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(700, activation='tanh')(decoded_layer_3)
    decoded = Dense(784, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder

def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q


def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g])
        return l1norm+l2norm+structpreserve+orthonormal
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt))))
    return q2

def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4

def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.tensordot(transz,z,axes=1)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5
#train_set,y=load_data()
#vtr=dnet.predict(train_set)
#vtr = vtr.reshape((len(vtr), np.prod(vtr.shape[1:])))
#x,y=extract_features(vtr,y)
#x_train=x
nev_trans,L,eig_val,eig_vec,dec_eigVal,W=eig_vec(x_train)
#np.save('eyalebdataforSb.npy',x_train)
#np.save('eyaleblabelsforSb.npy',y)
alpha_trans1=np.load('./datasets/EYALEB/Sb_eyaleb.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32')
encoder,encoding_dim,autoencoder=arch()
autoencoder.compile(tf.keras.optimizers.Adam(lr=1e-3), loss=tf.keras.losses.MeanSquaredError())
autoencoder.fit(x_train,x_train,epochs=100,shuffle=True,batch_size=batchSize)
encoder.save_weights('./newcheckpoints/nchk_eyaleb/nE_Eyaleb.ckpt')

print('training from scratch')
encoder.load_weights('./newcheckpoints/nchk_eyaleb/nE_Eyaleb.ckpt')
g=0
encoder.compile(tf.keras.optimizers.Adadelta(lr=1e-3, rho=0.95, decay=1e-4), loss=asl())
acc=0
for t in range (1,900,1):
    g=0
    for l in range (0,len(x_train),batchSize):
        encoder.fit(x_train[l:l+batchSize],nev_trans[g][:,batchSize-encoding_dim-1:batchSize-1],epochs=1,shuffle=False,batch_size=batchSize)
        g=g+1
    z_test=encoder.predict(x_train)
    k_means = KMeans(n_clusters=k_cluster, init='k-means++')
    ic= k_means.fit(z_test).cluster_centers_
    assignment = assign_func(z_test, ic)
    predictedlabels = np.argmax(assignment, axis=1)
    rectified_label, num=rectify_label(predictedlabels, y)
    if rectified_label is not None:
        print('acc at',t, 'epoch is:',acc)
        if acc<num:
            acc=num
            encoder.save_weights('./newcheckpoints/nchk_eyaleb/nsdsc_Eyaleb.ckpt')
            np.save('./newcheckpoints/nchk_eyaleb/nsdsc_icEyaleb.npy',ic)
            nmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
            f1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
            prec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
    else:
        continue
print(acc,nmi,f1,prec)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


5/5 [==============================] - 2s 307ms/step - loss: 0.0018
Epoch 2/100
5/5 [==============================] - 1s 183ms/step - loss: 0.0012
Epoch 3/100
5/5 [==============================] - 1s 178ms/step - loss: 9.8321e-04
Epoch 4/100
5/5 [==============================] - 1s 173ms/step - loss: 8.1522e-04
Epoch 5/100
5/5 [==============================] - 1s 172ms/step - loss: 7.0274e-04
Epoch 6/100
5/5 [==============================] - 1s 173ms/step - loss: 6.2408e-04
Epoch 7/100
5/5 [==============================] - 1s 168ms/step - loss: 5.6188e-04
Epoch 8/100
5/5 [==============================] - 1s 170ms/step - loss: 5.1139e-04
Epoch 9/100
5/5 [==============================] - 1s 172ms/step - loss: 4.7121e-04
Epoch 10/100
5/5 [==============================] - 1s 172ms/step - loss: 4.3877e-04
Epoch 11/100
5/5 [==============================] - 1s 170ms/step - loss: 4.1223e-04
Epoch 12/100
5/5 [==============================] - 1s 168ms/step - loss: 3.9052e-04
Epoch 13/

/usr/local/lib/python3.11/dist-packages/keras/optimizers/legacy/adadelta.py:82: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
acc at 185 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 7ms/step
acc at 186 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 7ms/step
acc at 187 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 7ms/step
acc at 188 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 7ms/step
acc at 189 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 8ms/step
acc at 190 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 8ms/step
acc at 191 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 11ms/step
acc at 192 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 9ms/step
acc at 193 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 8ms/step
acc at 194 epoch is: 0.9658436213991769
76/76 [==============================] - 1s 11ms/step
acc at 195 epoch 